In [1]:
from scipy import stats

In [2]:
from scipy.stats import spearmanr

In [3]:
from scipy.stats import ConstantInputWarning

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.pipeline.evaluation.evaluate_regression import get_rankings, average_spearman, custom_train_test_split

In [14]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from category_encoders import OneHotEncoder
from src.pipeline.pipeline_transformers import OneHotEncoderTransformer2, Node2VecEmbedding, ColumnKeeper
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from src.features.encoder_utils import load_graph

from src.configuration import load_dataset
from src import configuration as config

factors = ["dataset", "model", "tuning", "scoring"]
new_index = "encoder"

# ---- load data...
DATA_DIR = Path( "../data/raw")

df_train = load_dataset(DATA_DIR / "dataset_train.csv")
X_train, X_test, y_train, y_test = custom_train_test_split(df_train, factors, "cv_score", train_size=0.75, random_state=42)
graph = load_graph(config.DATA_DIR / "external/graphs/encodings_graph.adjlist")

embedding = Node2VecEmbedding(graph)

# ---- predict
dummy_pipe = Pipeline([("embedding", embedding),
                        ("keeper", ColumnKeeper(columns=["node2vec_embedding_dim1", "node2vec_embedding_dim2"])),
                        ("model", LinearRegression())])

print(dummy_pipe.named_steps)

y_pred = pd.Series(dummy_pipe.fit(X_train, y_train).predict(X_test), index=y_test.index, name="cv_score_pred")
df_pred = pd.concat([X_test, y_test, y_pred], axis=1)

# ---- convert to rankings and evaluate
rankings_test = get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
rankings_pred = get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
print(average_spearman(rankings_test, rankings_pred))

{'model': LinearRegression()}


ValueError: could not convert string to float: 'LR'